### Summary

This will break down the file in the following steps:
1. Each "END GO" is treated as a unique SQL
2. look at each line and remove comments (--)
3. Look at comments block then remove (/* */)
4. Look for keywords like FROM, JOIN , UPDATE, INTO, CREATE TABLE then grab the next word (Database.Schema.Name or Schema.Name)
    (not necessary to find relationships, so all will be just using the RELY_ON relationship.)
5. Concatenating into a single dataframe and output to CSV
6. Ingest into Graph Database

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import re
import math
from tqdm import tqdm
import nbformat

In [2]:
%run "0_Configuration.ipynb"

input_script_view =  1.INPUT/DATAWAREHOUSE/ViewScript.sql
input_script_sp =  1.INPUT/DATAWAREHOUSE/SPsScript.sql
input_script_table =  1.INPUT/DATAWAREHOUSE/tableScript.sql
graph_ingestion_view =  3.OUTPUT_GRAPH/graph_input_vw.csv
graph_ingestion_sp =  3.OUTPUT_GRAPH/graph_input_sp.csv
graph_ingestion_sp2 =  3.OUTPUT_GRAPH/graph_input_sp2.csv
graph_ingestion_table =  3.OUTPUT_GRAPH/graph_input_table.csv


### Import Files
#### We will use 1 files from 1.INPUT folder
1A. Stored Procedure SQL files

In [3]:
def read_raw_sql_view(sql_input, regex_str):
    str_found  = re.findall(regex_str, sql_input)
  #  print("viewname found", str_found)
    df = pd.DataFrame (str_found, columns = ['SP_SCHEMA', 'SP_NAME'])
    df['SYNTAX'] = sql_input
    return df


def final_table_name_cleansing(text):
    if text==text:
            input_str = text.replace('FROM', '').replace('JOIN', '').replace(']', '').replace('[', '').replace(' ', '')
    else:
        input_str = ""
    return input_str

In [4]:
# Open and read the file as a single buffer
fd = open(input_script_sp, 'r', encoding="utf-16")
sqlFile = fd.read()
fd.close()

In [5]:
#This convert the raw SQL file into a Dataframe for our later REGEX manipulation

df = pd.DataFrame()
i= 1

for sql_statement in re.split(r'CREATE\s+PROC', sqlFile):
    i = i + 1
    concat_sql = ""
    sql_statement = sql_statement.upper()
    #print(i, sql_statement)
    for line in sql_statement.split("\n"):
        #Remove anythign on the right of the comment
        if not "/" in line: 
            q = line.split("--")[0]
            concat_sql = concat_sql + " " + q
       # print(line)
        

    concat_sql = concat_sql.replace('\t', ' ').replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ")
    concat_sql = re.sub(r"\/\*[\s\S]*?\*\/|([^:]|^)\/\/.*$", "", concat_sql)
    
    #print(concat_sql)
    
    df = pd.concat([df, read_raw_sql_view("CREATE PROC " + concat_sql, r"(?ims)\b(?:CREATE\s+PROC)\s+(\[?\w+\]?)\.(\[?\w+\]?)")], ignore_index=True )

df['SP_SCHEMA']=df['SP_SCHEMA'].apply(lambda x: final_table_name_cleansing(x))
df['SP_NAME']=df['SP_NAME'].apply(lambda x: final_table_name_cleansing(x))

df.apply(lambda x: x.astype(str).str.upper())
df['last_element'] =  df['SP_NAME'].str.split('_').str[-1]


# This is where you set logics to exclude certain SP by name
exclusion_list = ['TMP', 'TEMP', 
                  'BCK', 'BKP', 'BACK',
                  'TEST', 'RSV']

exclusion_list_v2 = ["_" + sub for sub in exclusion_list]

df['EXCLUSION_v1'] = df['SP_NAME'].apply(lambda x: any([k in x for k in exclusion_list_v2]))
df['EXCLUSION_v2'] = df['last_element'].apply(lambda x: any([k in x for k in exclusion_list]))

def find_suffix_digits(stringInput):
    return bool(re.search(r"_[\d]{4,8}$", stringInput))

df['EXCLUSION_digit'] = df['SP_SCHEMA'].apply(lambda x: find_suffix_digits(x))
df['EXCLUSION_utils'] = df.SP_SCHEMA.str.contains("util")

df['EXCLUSION'] = df.EXCLUSION_v1 | df.EXCLUSION_v2 | df.EXCLUSION_digit | df.EXCLUSION_utils

df = df[['SP_SCHEMA', 'SP_NAME', 'SYNTAX', 'EXCLUSION']]

filterlist = ['TCDT_TO_TRANSFERENCE_LOG_RAW',
    'PF_HKMN_SP_DIM_TABLE',
    'PF_HKWE_SP_BASIC_CALC_METRICS',
    'PF_HKWE_SP_DIM_TX_DETAIL',
    'PF_HKWE_SP_DIM_TX_ITEM',
    'PF_HKWE_SP_DIM_TX_PROMOTION',
    'PF_HKWE_SP_DIM_TX_STORE',
    'PF_HKWE_SP_DIM_TX_UPSCALE',
    'PF_HKWE_SP_HIERARCHY_DASHBOARD',
    'PF_HKWE_SP_INSERT_MEM_CNT_YEARLY_L13',
    'PF_HKWE_SP_INSERT_MEM_CNT_YEARLY_L52',
    'PF_HKWE_SP_INSERT_MEM_CNT_YEARLY_TW_L4W',
    'PF_HKWE_SP_STG_ALL_PERIODE',
    'PF_HKWE_SP_STG_MEM_CNT_YEARLY_YTD',
    'PF_HKWE_SP_STG_MONTHLY',
    'PF_HKWE_SP_STG_MONTHLY_SEGMENT',
    'PF_HKWE_SP_STG_SEGMENT_BASE',
    'PF_HKWE_SP_STG_WEEKLY',
    'PF_HKWE_SP_STG_YEARLY',
    'PF_HKWE_SP_STG_YEARLY_SEGMENT',
    'PF_HKWE_SP_TX_FINAL']

filterlist = ['TCDT_TO_TRANSFERENCE_LOG_RAW']

df = df[df['SP_NAME'].isin(filterlist)]

df

#### This is what the output dataframe should look like to feed into the Regex Engine

### 2. Main ETL Codes
#### Now using REGEX to identify the SP-Table relationships
#### We will use only the SP Dataframe we got earlier

In [6]:

#This function is used to loop complex pattern like "INSERT INTO FROM SELECT * FROM xxx"
#Since there can be multiple SELECT FROM tables, Will start from the most outer shell, then move inner
#If There is a CREATE TABLE/INSERT TABLE statement, then will get from there
def regex_part_analyser(sql_input, regex_type, regex_str, df_master):
    str_found  = re.findall(regex_str, sql_input)
    str_cat=str_found

    df = pd.DataFrame (str_cat, columns = ['TABLE_FULL_NAME'])
    df['REGEX_TYPE'] = regex_type
    return df

def regex_manipulation(sql_input, regex_type, regex_str, opt_df=None, opt_col=None):
    output = re.findall(regex_str, sql_input)
    output = list(dict.fromkeys(output))
    
    if opt_df is not None:
            output = [x for x in output if x not in opt_df[opt_df.REGEX_TYPE==opt_col]['TABLE_FULL_NAME'].tolist()]

    df = pd.DataFrame (output, columns = ['TABLE_FULL_NAME'])
    df['REGEX_TYPE'] = regex_type
    
    return df

def isNaN(string):
    return string != string



def regex_main_analyser( index, rowInput, sql_Input):
    
    # remove the /* */ comments
    q = re.sub(r"/\*[^*]*\*+(?:[^*/][^*]*\*+)*/", "", sql_Input)

    #remove the */ and /* comments
    q  = re.sub(r"[^*]*\*+(?:[^*/][^*]*\*+)*/", "", q)
    q  = re.sub(r"\/\*[^,]*$", "", q)
    q  = q.replace('"','')
    q  = q.upper()
    df = pd.DataFrame()
    #print(q)
    
    #Splitting the Source Code into multiple SQL statement by ';'
    list_sql_statement = q.split(';')
    
    
    for sql_statement in list_sql_statement:

        #print(sql_statement)
        #1 components TABLE string database.schema.table
        df = pd.concat([df, regex_part_analyser(sql_statement, "SELECT_FROM", 
                                                         r"(?ims)\b(?:FROM|JOIN)\s+([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?))(?:,|\)|;| |$){1}", df)])
      
        df = pd.concat([df, regex_part_analyser(sql_statement, "SELECT_FROM", 
                                                         r"(?ims)\b(?:FROM|JOIN)\s+(?:[\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?\.[\[]?\w+[\]]?))\s*\w*\s*,\s*([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?\.[\[]?\w+[\]]?))(?:,|\)|;| |$){1}", df)])   
    
    
        df = pd.concat([df, regex_part_analyser(sql_statement, "INSERT_UPDATE", 
                                                         r"(?ims)\b(?:UPDATE|INTO|TABLE)\s+([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?))(?:,|\)|;| |$){1}", df)])
 
        df = pd.concat([df, regex_part_analyser(sql_statement, "INSERT_UPDATE", 
                                                         r"(?ims)\b(?:UPDATE|INTO|TABLE)\s+(?:[\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?\.[\[]?\w+[\]]?))\s*\w*\s*,\s*([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?\.[\[]?\w+[\]]?))(?:,|\)|;| |$){1}", df)])   
    

        df = pd.concat([df, regex_part_analyser(sql_statement, "SELECT_FROM", 
                                                         r"(?ims)\b(?:,)\s+(?:[\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?))", df)])
                
            
        df = pd.concat([df, regex_manipulation(sql_statement, "INSERT_UPDATE", 
                                               r"(?ims)(?:UPDATE|STATISTICS)\s+([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?))")])  

        df = pd.concat([df, regex_manipulation(sql_statement, "EXEC",
                                               r"(?ims)\b(?:EXEC|EXECUTE)\s+([\[]?[\w-]+(?:[\]]?\.[\[]?\w+[\]]?))")])

    df = df.drop_duplicates()  
    df['SP_SCHEMA'] =rowInput['SP_SCHEMA'] 
    df['SP_NAME'] =rowInput['SP_NAME'] 
  # df['SYNTAX'] =rowInput['SYNTAX'] 
    df['VW_INDEX'] = index
    df['SYNTAX_WORDCOUNT'] = len(q.split())
        
    return df

In [7]:

df_cat = pd.DataFrame()    

print(f"Total count of {len(df)} SP in the file")
for index, row in tqdm(df.iterrows()):
    
    #This will put the SQL Syntax into the main regex code analyser to identify table names
    if not isNaN(row['SYNTAX']):
        df_cat = pd.concat([ df_cat, regex_main_analyser(index, row, row['SYNTAX'])])
            
print("COMPLETED : Graph Import File saved to graph_input_sp.csv")

Total count of 1231 SP in the file


1231it [36:46,  1.79s/it]

COMPLETED : Graph Import File saved to graph_input_sp.csv


### Export the results into 3. OUTPUT_GRAPH
#### We will add back the remaining SP Name that have not been matched into the Output csv

In [8]:
df_cat2 = df_cat.copy()

df_cat2['regex_type_value'] = df_cat2['REGEX_TYPE'].map({'SELECT_FROM': 0, 'EXEC': 0, 
                                                        'INSERT_UPDATE': 1})

df_cat2['regex_sum_value'] = df_cat2.groupby(['TABLE_FULL_NAME', 'VW_INDEX']).regex_type_value.transform(np.sum)

df_cat3 = df_cat2.copy()
df_cat2 = df_cat2[(df_cat2.regex_sum_value == 1) & (df_cat2.REGEX_TYPE == 'INSERT_UPDATE')]
df_cat3 = df_cat3[(df_cat3.regex_sum_value == 0)]

df_final = pd.concat([df_cat2, df_cat3], ignore_index=True).drop_duplicates()

df_final

,TABLE_FULL_NAME,REGEX_TYPE,SP_SCHEMA,SP_NAME,VW_INDEX,SYNTAX_WORDCOUNT,regex_type_value,regex_sum_value
0,[ANALYSE].[ANFIELD_INGESTION_SUMMARY],INSERT_UPDATE,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,1,2672,1,1
1,ANALYSE.ANFIELD_INGESTION_SUMMARY,INSERT_UPDATE,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,1,2672,1,1
2,[ANALYSE].[ANFIELD_INGESTION_SUMMARY],INSERT_UPDATE,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY_ALP,2,2958,1,1
3,ANALYSE.ANFIELD_INGESTION_SUMMARY,INSERT_UPDATE,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY_ALP,2,2958,1,1
4,[ANALYSE].[ANFIELD_INGESTION_SUMMARY],INSERT_UPDATE,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY_IKEA,3,955,1,1
...,...,...,...,...,...,...,...,...
12981,[PUBLISH].[ANFIELD_LOY_MEMBER_ACCOUNT],SELECT_FROM,PUBLISH_YUU,YUUTB_RFM_TAB_4_SP_MONTHENDUPDATE,1225,1665,0,0
12982,[PUBLISH].[RFM_SEGMENT_MXHK],SELECT_FROM,PUBLISH_YUU,YUUTB_RFM_TAB_4_SP_MONTHENDUPDATE,1225,1665,0,0
12983,[PUBLISH_YUU].[YUUTB_RFM_TAB_4_DATA],SELECT_FROM,PUBLISH_YUU,YUUTB_RFM_TAB_4_SP_MONTHENDUPDATE_AGGR,1226,212,0,0
12984,UTIL.DROP_TBL_IF_EXISTS,EXEC,UTIL,CREATE_OR_REPLACE_TBL,1228,73,0,0


In [9]:
#This checks if there are any left out scripts without TABLE name
delta_df = pd.merge(df, df_final, how='left', left_on=['SP_SCHEMA','SP_NAME'], right_on = ['SP_SCHEMA','SP_NAME'])
delta_df.drop(columns=["SYNTAX"], inplace=True)
delta_df['TABLE_FULL_NAME']=delta_df['TABLE_FULL_NAME'].apply(lambda x: final_table_name_cleansing(x))
delta_df = delta_df.applymap(lambda s: s.upper() if type(s) == str else s)

delta_df.to_csv(graph_ingestion_sp)


In [10]:
delta_df

,SP_SCHEMA,SP_NAME,EXCLUSION,TABLE_FULL_NAME,REGEX_TYPE,VW_INDEX,SYNTAX_WORDCOUNT,regex_type_value,regex_sum_value
0,ANALYSE,ADF_RAISE_EXCEPTION,False,,NaN,NaN,NaN,NaN,NaN
1,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,False,ANALYSE.ANFIELD_INGESTION_SUMMARY,INSERT_UPDATE,1.0,2672.0,1.0,1.0
2,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,False,ANALYSE.ANFIELD_INGESTION_SUMMARY,INSERT_UPDATE,1.0,2672.0,1.0,1.0
3,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,False,PREPARE.ANFIELD_ALP_LOCATION_SYNC,SELECT_FROM,1.0,2672.0,0.0,0.0
4,ANALYSE,ANFIELD_TO_INGESTION_SUMMARY,False,PREPARE.ANFIELD_ALP_PRODUCT_SYNC,SELECT_FROM,1.0,2672.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
13099,PUBLISH_YUU,YUUTB_RFM_TAB_4_SP_MONTHENDUPDATE_AGGR,False,PUBLISH_YUU.YUUTB_RFM_TAB_4_DATA,SELECT_FROM,1226.0,212.0,0.0,0.0
13100,UTIL,ANFIELD_LOY_PARAMETERS,False,,NaN,NaN,NaN,NaN,NaN
13101,UTIL,CREATE_OR_REPLACE_TBL,False,UTIL.DROP_TBL_IF_EXISTS,EXEC,1228.0,73.0,0.0,0.0
13102,UTIL,DROP_TBL_IF_EXISTS,False,,NaN,NaN,NaN,NaN,NaN
